# Base Experiments

In [1]:
from recbole.config import Config
from recbole.evaluator.evaluator import Evaluator
import pandas as pd
import pickle
import numpy as np

c:\Users\rbt938\Anaconda3\envs\fairreceval\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
import time
import torch

In [3]:
filenames_of_best_model = [
 'filename_best_model_for_Amazon_Luxury_Beauty.pickle',
 'filename_best_model_for_lastfm.pickle',
 'filename_best_model_for_ml-1m.pickle',
  'filename_best_model_for_book-crossing.pickle',
 'filename_best_model_for_Amazon_Industrial_and_Scientific.pickle',
'filename_best_model_for_Amazon_Digital_Music.pickle',
 ]

In [ ]:
result_dict = dict()
k=10
for filename in filenames_of_best_model:
    dataset = filename.replace('filename_best_model_for_',"").replace(".pickle","")
    print(f"Doing {dataset}")
    result_dict[dataset] = dict()
    
    config = Config(
                model="Pop", 
                dataset=dataset, 
                config_file_list=["../RecBole/recbole/properties/overall.yaml"],
                config_dict={"topk": [k],
                "metrics":["RelMetrics","FairWORel","IID_AID"]
                }
                )

    with open(f"../train_val_test/{dataset}_test.pickle","rb") as f:
        data = pickle.load(f)
    test = pd.DataFrame(data)

    item_id = config.final_config_dict["ITEM_ID_FIELD"]

    test = test.groupby("user_id")\
        .agg(lambda x: [x for x in x])\
        [item_id]
    evaluator = Evaluator(config)

    with open("../results/"+filename,"rb") as f:
        models = pickle.load(f)

    #non-pop model
    for model in models:

        model_name = model.split("-")[0]
        print(model_name)
        with open(f"../hendrix/struct/struct_{dataset}_"+model,"rb") as f:
            struct = pickle.load(f)

        item_matrix = struct.get('rec.items')
        rec_mat = struct.get('rec.topk')
        struct.set("data.pos_items",test.apply(np.array).values)

        #slice item_matrix and relevance_matrix
        updated_item_matrix = item_matrix[:,:k]
        updated_rec_mat = torch.cat((rec_mat[:,:k], rec_mat[:,-1:]), axis=1)

        struct.set("rec.items",updated_item_matrix)
        struct.set("rec.topk",updated_rec_mat)

        struct.set("data.pos_items",test.apply(np.array).values)

        start_time = time.time()
        result = evaluator.evaluate(struct)
        print("total time taken: ", time.time() - start_time)
        result_dict[dataset][model_name] = result
        print(result)
        with open(f"../results/base/base_{dataset}_{model_name}.pickle","wb") as f:
            pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)

    #pop result
    model_name = "Pop"
    print(model_name)
    
    list_file = os.listdir("../struct/")
    file_for_dataset = [x for x in list_file if dataset in x]
    assert len(file_for_dataset) == 1

    with open("../struct/"+file_for_dataset[0],"rb") as f:
        struct = pickle.load(f)

    item_matrix = struct.get('rec.items')
    rec_mat = struct.get('rec.topk')
    #slice item_matrix and relevance_matrix
    updated_item_matrix = item_matrix[:,:k]
    updated_rec_mat = torch.cat((rec_mat[:,:k], rec_mat[:,-1:]), axis=1)

    struct.set("rec.items",updated_item_matrix)
    struct.set("rec.topk",updated_rec_mat)

    struct.set("data.pos_items",test.apply(np.array).values)
    start_time = time.time()
    result = evaluator.evaluate(struct)
    print("total time taken: ", time.time() - start_time)

    result_dict[dataset][model_name] = result
    print(result)
    with open(f"../results/base/base_{dataset}_{model_name}.pickle","wb") as f:
        pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)